In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

/home/quile/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
X_train = mnist.train.images.reshape(-1, 28, 28, 1)
Y_train = mnist.train.labels
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=2)
X_test = mnist.test.images.reshape(-1, 28, 28, 1)
Y_test = mnist.test.labels

In [4]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [5]:
model = keras.models.Sequential()

model.add(keras.layers.Conv2D(filters = 32, kernel_size = (5,5),padding = 'Valid', 
                              activation ='relu', input_shape = (28,28,1)))
model.add(keras.layers.MaxPool2D(pool_size=(2,2)))
model.add(keras.layers.Conv2D(filters = 32, kernel_size = (5,5),padding = 'Valid', 
                              activation ='relu'))
model.add(keras.layers.MaxPool2D(pool_size = (2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(120, activation = 'relu'))
model.add(keras.layers.Dense(84, activation = 'relu'))
model.add(keras.layers.Dense(10, activation = 'softmax'))

# model = keras.models.Sequential()

# model.add(keras.layers.Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
#                  activation ='relu', input_shape = (28,28,1)))
# model.add(keras.layers.Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
#                  activation ='relu'))
# model.add(keras.layers.MaxPool2D(pool_size=(2,2)))
# model.add(keras.layers.Dropout(0.25))


# model.add(keras.layers.Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
#                  activation ='relu'))
# model.add(keras.layers.Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
#                  activation ='relu'))
# model.add(keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
# model.add(keras.layers.Dropout(0.25))


# model.add(keras.layers.Flatten())
# model.add(keras.layers.Dense(256, activation = "relu"))
# model.add(keras.layers.Dropout(0.5))
# model.add(keras.layers.Dense(10, activation = "softmax"))

In [6]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 24, 24, 32)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 32)          25632     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 120)               61560     
_________________________________________________________________
dense_2 (Dense)              (None, 84)                10164     
__________

In [7]:
batch_size = 32
epochs = 10
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [8]:
# history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=epochs, batch_size=batch_size, callbacks=[learning_rate_reduction])

In [9]:
datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=10,
        zoom_range = 0.1,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=False,
        vertical_flip=False)
datagen.fit(X_train)
history = model.fit_generator(datagen.flow(X_train,Y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_val,Y_val), 
                              steps_per_epoch=X_train.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])

Epoch 1/10
1546/1546 [==============================] - 69s 45ms/step - loss: 0.9085 - acc: 0.6864 - val_loss: 1.4207 - val_acc: 0.9105
Epoch 2/10
1546/1546 [==============================] - 68s 44ms/step - loss: 0.3063 - acc: 0.9041 - val_loss: 1.3205 - val_acc: 0.9169
Epoch 3/10
1546/1546 [==============================] - 69s 44ms/step - loss: 0.2208 - acc: 0.9304 - val_loss: 1.0836 - val_acc: 0.9318
Epoch 4/10
1546/1546 [==============================] - 70s 45ms/step - loss: 0.1831 - acc: 0.9415 - val_loss: 0.7464 - val_acc: 0.9525
Epoch 5/10
1546/1546 [==============================] - 68s 44ms/step - loss: 0.1580 - acc: 0.9494 - val_loss: 0.5345 - val_acc: 0.9664
Epoch 6/10
1546/1546 [==============================] - 70s 46ms/step - loss: 0.1419 - acc: 0.9548 - val_loss: 0.8746 - val_acc: 0.9455
Epoch 7/10
1546/1546 [==============================] - 68s 44ms/step - loss: 0.1294 - acc: 0.9593 - val_loss: 1.2049 - val_acc: 0.9240
Epoch 8/10
1546/1546 [==========================

In [ ]:
model.evaluate(X_test, Y_test, batch_size=32)

 4544/10000 [============>.................] - ETA: 2s

In [ ]:
# model.save('MNIST_model.h5')

In [ ]:
# model = keras.models.load_model('MNIST_model.h5')

In [ ]:
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['acc'], color='b', label="Training accuracy")
ax[1].plot(history.history['val_acc'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)

In [ ]:
Y_pred = model.predict(X_test)
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
Y_true = np.argmax(Y_test,axis = 1) 

In [ ]:
wrong_case = [index for (index, (y_pred, y_true)) in enumerate(zip(Y_pred_classes, Y_true)) if y_pred != y_true]
print(len(wrong_case))
fig, ax = plt.subplots(2,3,sharex=True,sharey=True)
for row in range(2):
    for col in range(3):
        index = wrong_case[row+col]
        ax[row,col].imshow(X_test[index][:,:,0])
        ax[row,col].set_title("{},{}".format(Y_pred_classes[index],Y_true[index]))
    
